## 导入tushare

In [65]:
import pandas as pd
import tushare as ts
print(ts.__version__)

1.2.77


## 设置token

In [5]:
ts.set_token('208f20a157fe345489034c653e6b5d3c85848600178dfab4d908e20a')

## 初始化pro接口

In [6]:
pro = ts.pro_api()

## 上证50成份股股票代码
由于tushare查看指数成分和权重的api积分要求较高，直接在东方财富网上（ https://data.eastmoney.com/other/index/sz50.html ）获取上证50成份股的代码

In [7]:
sz50_codes = ['600000', '600028', '600030', '600031', '600036', '600048', '600050', '600104', '600196', '600276', '600309', '600436',
              '600438', '600519', '600547', '600570', '600585', '600588', '600690', '600745', '600809', '600837', '600887', '600893',
              '600900', '601012', '601066', '601088', '601138', '601166', '601211', '601288', '601318', '601336', '601398', '601601',
              '601628', '601633', '601668', '601688', '601728', '601857', '601888', '601899', '601919', '601995', '603259', '603288',
              '603501', '603986']
print(len(sz50_codes))

51


## 查看上证50成份股信息
获取股票的Tushare代码（ts_code）以及股票名称、行业、上市日期等信息，保存到csv文件

In [70]:
all_stocks_info = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
sz50_info = all_stocks_info[all_stocks_info['symbol'].isin(sz50_codes)].reset_index(drop=True)
print(len(sz50_info))
print(sz50_info)
sz50_info.to_csv('../data/上证50股票信息.csv')

50
      ts_code  symbol  name area industry list_date
0   600000.SH  600000  浦发银行   上海       银行  19991110
1   600028.SH  600028  中国石化   北京     石油加工  20010808
2   600030.SH  600030  中信证券   深圳       证券  20030106
3   600031.SH  600031  三一重工   北京     工程机械  20030703
4   600036.SH  600036  招商银行   深圳       银行  20020409
5   600048.SH  600048  保利发展   广东     全国地产  20060731
6   600050.SH  600050  中国联通   北京     电信运营  20021009
7   600104.SH  600104  上汽集团   上海     汽车整车  19971125
8   600196.SH  600196  复星医药   上海     化学制药  19980807
9   600276.SH  600276  恒瑞医药   江苏     化学制药  20001018
10  600309.SH  600309  万华化学   山东     化工原料  20010105
11  600436.SH  600436   片仔癀   福建      中成药  20030616
12  600438.SH  600438  通威股份   四川     电气设备  20040302
13  600519.SH  600519  贵州茅台   贵州       白酒  20010827
14  600547.SH  600547  山东黄金   山东       黄金  20030828
15  600570.SH  600570  恒生电子   浙江     软件服务  20031216
16  600585.SH  600585  海螺水泥   安徽       水泥  20020207
17  600588.SH  600588  用友网络   北京     软件服务  20010518
18  60069

## 获取上证50成份股历史数据
提取股票信息的ts_code列，转成api要求的逗号分隔字符串，然后通过daily接口查询，返回结果按股票编号排序。

In [39]:
query_codes = sz50_info['ts_code'].tolist()
query_codes = ','.join(query_codes)
print(query_codes.count(','))
print(query_codes)
sz50_prices = pro.daily(ts_code=query_codes, start_date='20090101', end_date='20211131')
sz50_prices = sz50_prices.sort_values(by=['ts_code', 'trade_date']).reset_index(drop=True)

# pd.set_option('display.max_rows', None)
print(sz50_prices.shape)

50


## 选取上证50中上市期限较长的股票作为交易股
通过

In [37]:
print(sz50_prices['ts_code'].value_counts())
print(len(sz50_prices['ts_code'].value_counts()))

ts_code    ts_code  
600000.SH  600000.SH    8
600028.SH  600028.SH    8
600030.SH  600030.SH    8
600031.SH  600031.SH    8
600036.SH  600036.SH    8
600048.SH  600048.SH    8
600050.SH  600050.SH    8
600104.SH  600104.SH    8
600196.SH  600196.SH    8
600276.SH  600276.SH    8
600309.SH  600309.SH    8
600436.SH  600436.SH    8
600438.SH  600438.SH    8
600519.SH  600519.SH    8
600547.SH  600547.SH    8
600570.SH  600570.SH    8
600585.SH  600585.SH    8
600588.SH  600588.SH    8
600690.SH  600690.SH    8
600745.SH  600745.SH    8
600809.SH  600809.SH    8
600837.SH  600837.SH    8
600887.SH  600887.SH    8
600893.SH  600893.SH    8
601012.SH  601012.SH    8
601066.SH  601066.SH    8
601088.SH  601088.SH    8
601138.SH  601138.SH    8
601166.SH  601166.SH    8
601211.SH  601211.SH    8
601288.SH  601288.SH    8
601318.SH  601318.SH    8
601336.SH  601336.SH    8
601398.SH  601398.SH    8
601601.SH  601601.SH    8
601628.SH  601628.SH    8
601633.SH  601633.SH    8
601668.SH  601668

In [69]:
# s1 = set(sz50_prices['ts_code'].value_counts().index.tolist())
# s2 = set(sz50_info['ts_code'].tolist())
# print(pro.daily(ts_code='600900.SH', start_date='20201201', end_date='20211212'))
dji = pd.read_csv('D:\Dev\!Workspace\Python\Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020\data\dow_30_2009_2020.csv')
print(dji)
print(dji['tic'].value_counts())
print(sorted(dji['tic'].value_counts().index.tolist()))

       Unnamed: 0  datadate  tic   prccd  ajexdi   prcod   prchd    prcld  \
0               0  20090102  AXP   19.33     1.0   18.57   19.52   18.400   
1               1  20090105  AXP   19.95     1.0   19.20   20.24   19.000   
2               2  20090106  AXP   21.07     1.0   20.30   21.38   20.010   
3               3  20090107  AXP   20.01     1.0   20.53   20.72   19.830   
4               4  20090108  AXP   20.04     1.0   19.80   20.17   19.400   
...           ...       ...  ...     ...     ...     ...     ...      ...   
87775      108896  20200811    V  197.77     1.0  197.82  202.89  196.335   
87776      108897  20200812    V  198.74     1.0  199.00  200.23  198.240   
87777      108898  20200813    V  197.58     1.0  197.13  199.94  197.030   
87778      108899  20200814    V  196.64     1.0  197.01  198.07  195.770   
87779      108900  20200817    V  199.43     1.0  197.30  200.46  196.710   

           cshtrd  
0      10955620.0  
1      16019130.0  
2      13819380

In [75]:
print(sz50_info[sz50_info['list_date'] < '20090101'])

      ts_code  symbol  name area industry list_date
0   600000.SH  600000  浦发银行   上海       银行  19991110
1   600028.SH  600028  中国石化   北京     石油加工  20010808
2   600030.SH  600030  中信证券   深圳       证券  20030106
3   600031.SH  600031  三一重工   北京     工程机械  20030703
4   600036.SH  600036  招商银行   深圳       银行  20020409
5   600048.SH  600048  保利发展   广东     全国地产  20060731
6   600050.SH  600050  中国联通   北京     电信运营  20021009
7   600104.SH  600104  上汽集团   上海     汽车整车  19971125
8   600196.SH  600196  复星医药   上海     化学制药  19980807
9   600276.SH  600276  恒瑞医药   江苏     化学制药  20001018
10  600309.SH  600309  万华化学   山东     化工原料  20010105
11  600436.SH  600436   片仔癀   福建      中成药  20030616
12  600438.SH  600438  通威股份   四川     电气设备  20040302
13  600519.SH  600519  贵州茅台   贵州       白酒  20010827
14  600547.SH  600547  山东黄金   山东       黄金  20030828
15  600570.SH  600570  恒生电子   浙江     软件服务  20031216
16  600585.SH  600585  海螺水泥   安徽       水泥  20020207
17  600588.SH  600588  用友网络   北京     软件服务  20010518
18  600690.S